In [ ]:
#@title Installieren der Libraries
#@markdown Dieser Abschnitt installiert alle nötigen Libraries.
!pip install xmltodict
!pip install entsoe-py
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install prophet
!pip install plotly
!pip install scikit-learn

In [ ]:
#@title Importieren der Libraries
#@markdown Dieser Abschnitt lädt die nötigen Libraries.
import requests
import pandas as pd
import xmltodict
from datetime import timedelta
from matplotlib import pyplot as plt
from entsoe import EntsoePandasClient
import seaborn as sns
import warnings
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error


warnings.filterwarnings("ignore")
pd.options.plotting.backend = "plotly"



In [ ]:
#@title Abrufen der Stromgrosshandelspreise durch den ENTSOE API
#@markdown Wir rufen die den API-Helfer ab um die Daten zu sammeln.\
#@markdown Hier werden die Daten beschrieben: https://transparency.entsoe.eu/content/static_content/Static%20content/knowledge%20base/data-views/transmission-domain/Data-view%20Day-ahead%20prices.html \
#@markdown **=> Durchschnittspreis des heute an der Börse gekauften Stroms für den folgenden Tag**

# Helfer
MYTOKEN="c81dca9c-4b53-4d16-9aa9-5674723d728a"
client = EntsoePandasClient(api_key=MYTOKEN)

# Abfrage Parameter
country_code='CH'
start = pd.Timestamp('20160101', tz='Europe/Brussels')
end = pd.Timestamp('20240630', tz='Europe/Brussels')

# Abfrage
df = client.query_day_ahead_prices(country_code, start=start, end=end).to_frame().reset_index()            .rename(columns={"index": "timestamp", 0: "price_ahead"})

# Wir erhalten stündliche Daten. Wir wollen aber tägliche Daten. Wir extrahieren das Datum...
df["date"] = pd.to_datetime(pd.to_datetime(df["timestamp"], utc=True).dt.date)
# ... und nehmen den Durchschnitt
df=df.groupby("date")['price_ahead'].mean().to_frame()

# Anzeigen als Tabelle
display(df.tail(5))

# Anzeigen als Liniendiagramm
fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df.index,
        y=df['price_ahead'],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    )

])

# Anpassen der Beschriftung
fig.update_layout(

    title="Stromgrosshandelspreis Schweiz",
    yaxis_title="Preis [EUR / MWh]",
    xaxis_title="Datum",
)

fig.show()
# Um die Abbildung im Notebook darzustellen, speichern wir sie als html
fig.write_html("price_dayahead.html")
FileLink("price_dayahead.html")


,price_ahead
date,
2024-06-25,49.352917
2024-06-26,54.712083
2024-06-27,58.962917
2024-06-28,42.198333
2024-06-29,34.558261


In [ ]:
#@title Model trainieren
#@markdown Wir unterteilen die Sequenz mit Stichtag 2019-10-01 in Trainings- und Testdaten und trainieren das Model auf den Trainingsdaten. \
#@markdown Das Problem welches wir zu lösen versuchen ist das der univariaten Vorhersage \


stichtag = '2019-10-01'

# Formatieren des Tabellenbeschriftung zum Standard
df['ds'] = df.index
df['y'] = df['price_ahead']

# Unterteilen der Daten in Trainings- und Testdaten anhand des Stichtags
df_train = df[df['ds'] < stichtag]
df_test = df[df['ds'] >= stichtag]

# Trainieren des Modells
m = Prophet(n_changepoints=0)
m.fit(df_train,)
#@markdown Mehr Informationen hier: https://facebook.github.io/prophet/docs/quick_start.html#python-api

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt617iags/zsta8r7l.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpt617iags/8lef2q83.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=8105', 'data', 'file=/tmp/tmpt617iags/zsta8r7l.json', 'init=/tmp/tmpt617iags/8lef2q83.json', 'output', 'file=/tmp/tmpt617iags/prophet_modelpwlabx1_/prophet_model-20241030122817.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:28:17 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:28:17 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
#@title Vorhersage: 4 Jahre in die Zukunft
#@markdown Wir lassen das Modell die nächsten 4 Jahre ab 2019-10-01 forecasten und stellen die Prognosen dar.
#@markdown * 2020 Corona
#@markdown * 2021 Grosse Nachfrage aus dem asiatischen Raum nach verflüssigtem Gas.
#@markdown * 2021 Kohleausstieg seitens Deutschlands
#@markdown * 2021 Unklare politische Lage rund um die russische Gaspipeline Nord Stream 2
#@markdown * 2022 Ukraine Krieg

# Vorhersagen der nächsten 365 * 4.8 Tage mit dem trainierten Modell
future = m.make_future_dataframe(periods=int(365*4.8))
forecast = m.predict(future)

# Anzeigen der echten Werte
fig = go.Figure([
    go.Scatter(x=df['ds'], y=df['y'], mode='lines', name='Echte Werte'),
# Anzeigen der Vorhersage
    go.Scatter(
        name='Forecast',
        x=forecast['ds'],
        y=forecast['yhat'],
        mode='lines',opacity=0.7
    ),
# Anzeigen der oberen Schranke
    go.Scatter(
        name='Upper Bound',
        x=forecast['ds'],
        y=forecast['yhat_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False,opacity=0.7
    ),
# Anzeigen der unteren Schranke
    go.Scatter(
        name='Lower Bound',
        x=forecast['ds'],
        y=forecast['yhat_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False,opacity=0.7
    )
])
# Anzeigen des Stichtags
fig.add_vline(x=pd.to_datetime(stichtag), line_width=3)

# Um die Abbildung im Notebook darzustellen, speichern wir sie als html
fig.write_html("price_dayahead_model1.html")
FileLink("price_dayahead_model1.html")


In [ ]:
#@title Vorhersage: 28 Tage in die Zukunft
#@markdown Wir lassen das Modell jeden Tag, den Preis 28 Tage später vorhersagen.

# Wir machen die Vorhersage mit dem Modell wie zuvor
future = m.make_future_dataframe(periods=int(365*5))
forecast = m.predict(future)

# Wir setzen das Datum als index
forecast.index = forecast['ds']
forecast['real'] = df['y']

# Wir nehmen den 8 Wochen Schnitt um die Fluktuationen zu reduzieren
forecast[['real_rolling','yhat_rolling']] = forecast[['real','yhat']].rolling(56, center=True).mean()
# Wir berechnen den Fehler unseres Modells for 4 Wochen...
forecast['error'] = (forecast['yhat_rolling'] - forecast['real_rolling']).shift(28)
# ...und fügen diesen Wert heute hinzu
# Im Umkehrschluss heisst das, dass wir anhand des Fehlers heute die Vorhersage in 4 Wochen anpassen
forecast['y_hat_adjusted'] = forecast['yhat'] - forecast['error']
forecast['yhat_upper_adjusted'] = forecast['yhat_upper'] - forecast['error']
forecast['yhat_lower_adjusted'] = forecast['yhat_lower'] - forecast['error']

# Anzeigen der echten Werte
fig = go.Figure([
    go.Scatter(x=df['ds'], y=df['y'], mode='lines', name='Echte Werte'),
    go.Scatter(
        name='Forecast',
        x=forecast['ds'],
        y=forecast['y_hat_adjusted'],
        mode='lines',opacity=0.7
    ),
# Anzeigen der oberen Schranke
    go.Scatter(
        name='Upper Bound',
        x=forecast['ds'],
        y=forecast['yhat_upper_adjusted'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False,opacity=0.7
    ),
# Anzeigen der unteren Schranke
    go.Scatter(
        name='Lower Bound',
        x=forecast['ds'],
        y=forecast['yhat_lower_adjusted'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False,opacity=0.7
    )
])
# Anzeigen des Stichtags
fig.add_vline(x=pd.to_datetime(stichtag), line_width=3)

# Um die Abbildung im Notebook darzustellen, speichern wir sie als html
fig.write_html("price_dayahead_model2.html")
FileLink("price_dayahead_model2.html")

In [ ]:
#@title Metriken
#@markdown Wir messen noch schnell zwei KPIs:
ds = forecast.loc[lambda x: x["ds"] > stichtag].dropna()
mape = mean_absolute_percentage_error(ds['real'], ds['y_hat_adjusted'])
mae = mean_absolute_error(ds['real'], ds['y_hat_adjusted'])
print(f"Mittlerer absoluter prozentualer Fehler (MAPE): {mape*100:.1f} [%]")
print(f"Mittlere absoluter Fehler (MAE): {mae:.1f} [EUR / MWh]")
#@markdown Je niedrige diese Werte, desto genauer / besser das Model.


Mittlerer absoluter prozentualer Fehler (MAPE): 41.8 [%]
Mittlere absoluter Fehler (MAE): 31.4 [EUR / MWh]
